In [1]:
from dotenv import find_dotenv
from dotenv import load_dotenv
print(find_dotenv())
succ = load_dotenv(find_dotenv())
if not succ:
    raise ValueError('load_dotenv FALSE')

/home/poyuan/workspace/rag30/.env


In [2]:
# llm setup
from llama_index.llms.openai import OpenAI
# set context for llm provider
llm = OpenAI(model="gpt-5-mini", temperature=0.0)

In [3]:
# document setup
# read document
from llama_index.core import SimpleDirectoryReader
source_dir = 'data/source'
reader = SimpleDirectoryReader(source_dir, required_exts=['.txt'])
documents = reader.load_data()

In [4]:
# simple node parser
## reference: https://developers.llamaindex.ai/python/framework/module_guides/loading/node_parsers/
## https://developers.llamaindex.ai/python/framework/module_guides/loading/node_parsers/modules/
from llama_index.core.node_parser import SentenceSplitter
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 20
SEPARATOR = " ?!;。！？；\n"

node_parser = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, separator=SEPARATOR)

nodes = node_parser.get_nodes_from_documents(
    documents, show_progress=False
)

In [5]:
len(nodes)

8

In [6]:
nodes[6]

TextNode(id_='f1306d52-0660-4cfc-9714-9f19edaca590', embedding=None, metadata={'file_path': '/home/poyuan/workspace/rag30/days/day05/data/source/lee.txt', 'file_name': 'lee.txt', 'file_type': 'text/plain', 'file_size': 14112, 'creation_date': '2025-10-07', 'last_modified_date': '2025-10-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6795090a-c0ee-47a9-9d79-9c61dc18709e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/poyuan/workspace/rag30/days/day05/data/source/lee.txt', 'file_name': 'lee.txt', 'file_type': 'text/plain', 'file_size': 14112, 'creation_date': '2025-10-07', 'last_modified_date': '2025-10-07'}, hash='59d094cda9d0399755e4648d82d090c921ba23d36deb890e58a4ab6c08aba7ab')

In [7]:
from llama_index.core.prompts.base import PromptTemplate
num_questions_per_chunk = 1

text_question_template = PromptTemplate(
    f"""你是一位專業的課程助教，你的任務是根據上課內容的片段，設計 {num_questions_per_chunk} 個問題來問學生。
問題風格以單一且直接的問句為主，並且需要在上課內容中找得到答案。
若是要求出複數題，則問題應該盡可能涵蓋不同的面向。

範例輸入:
以下是上課內容片段：
---------------------
"業設計成這樣呢\n為什麼作業要跑三個小時呢\n我只允許這個作業三分鐘\n就應該要>跑完\n我們有當然可以把這些特別花時間\n需要特別花時間訓練的作業拿掉\n但是我還是選>擇在這門課裡面\n保留那一些\n需要一定訓練時間的作業\n因為焦躁的等待人工智慧訓練的結果\n迷茫的調參數\n不知道會不會成功\n這個就是人工智慧的醍醐味\n所以大家需要學習\n在迷茫中前進\n這個就是模型的訓練\n我們特別把這一部分保留在課程裡面\n讓你體驗說\n模型訓練不出來的焦躁\n到底是什麼樣的感覺\n而且我必須要強調啊\n什麼三四個小時的等待訓練時間\n真的不算什麼\n我們過去有很多作業訓練時間都是\n至少三天起跳\n你要至少訓練三天\n你才能夠拿到成績\n那我知道說很多同學在做作業的時候\n往往你都在實現\n最後一天才開始做作業\n但那一種啊\n需要訓練三天以上的模型\n你只在前一天才開始做作業\n你是絕不可能完成的\n這個時候我告訴你\n你唯一可以做的事情\n就是放棄這樣子\n還好我們這堂課裡面呢\n現在是沒有需要訓練一天以上的作業啦\n我們把那種\n特別需要花時間訓練的作業\n還是拿掉了\n只保留了需要訓練三四個小時的作業\n但我只想要強調說\n三四個小時的訓練時間\n真的不算什麼\n如果你要真正用大量的資料\n大規模的訓練模型\n訓練個數週\n其實都是常見的事情\n而這一些\n需要一點訓練時間的作業\n它的定位就像是預防針\n幫助你在未來面對更大的挑戰\n幫助你在未來面對挑戰的時候\n做好心理準備\n另外呢\n鼓勵大家如果有空的話\n可以先看一些線上的錄影進行預習\n那假設你對於生成式AI一無所知的話\n那你可以先看生成式AI導論2024的課程\n那如果2024的課程看完\n你想要進一步了解\n這些AI是怎麼被訓練出來的"
---------------------
範例輸出:
什麼是深度學習的醍醐味？

輸入:
以下是上課內容片段：
---------------------
{{context_str}}
---------------------{{query_str}}
"""
)

text_qa_template = PromptTemplate("""以下是背景資訊。
---------------------
{context_str}
---------------------
僅根據上述背景資訊，而非先驗知識，回答下列問題。
問題：{query_str}
答案：""")

question_gen_query = '輸出:\n'

In [8]:
text_qa_template

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='以下是背景資訊。\n---------------------\n{context_str}\n---------------------\n僅根據上述背景資訊，而非先驗知識，回答下列問題。\n問題：{query_str}\n答案：')

In [9]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [10]:
dataset_generator = RagDatasetGenerator(
    nodes,
    llm=llm,
    num_questions_per_chunk=num_questions_per_chunk,
    text_question_template = text_question_template,
    text_qa_template = text_qa_template,
    question_gen_query = question_gen_query,
    show_progress=True,
)

In [11]:
len(dataset_generator.nodes)

8

In [12]:
rag_dataset = dataset_generator.generate_dataset_from_nodes()

/home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 38%|███████████████████████████████▉                                                     | 3/8 [00:07<00:10,  2.00s/it]2025-10-07 16:59:49,467 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-07 16:59:49,469 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.44s/it]


In [13]:
df = rag_dataset.to_pandas()
df.head(3)

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,這門課希望學生從使用者變成什麼？,[各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工智...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini)
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,[第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握AI...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini)
2,哪個軟體被用來合成講師的聲音？,[也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的\...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini)


In [14]:
import os
DEST_DIR = os.path.join('data', 'deliverables')
os.makedirs(DEST_DIR, exist_ok=True)
save_file_path = os.path.join(DEST_DIR, 'lee_course0_rag_dataset.csv')
df.to_csv(save_file_path, index=False)

# reload

In [15]:
import pandas as pd
import os

In [16]:
df = pd.read_csv(os.path.join('data', 'deliverables', 'lee_course0_rag_dataset.csv'))
df.head(3)

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,這門課希望學生從使用者變成什麼？,['各位同學大家好 我們來上課吧\n剛才只是用Google的VO3\n這個可以生成影片的人工...,理解者，甚至是創造者。,ai (gpt-5-mini),ai (gpt-5-mini)
1,剛才用來生成倒數計時影片和讓人長出翅膀的影片是使用哪個工具？,['第一種\n如果你未來想要更深入人工智慧領域\n這門可絕對是很好的入門\n當然要完全掌握A...,答案：Google 的 VO3,ai (gpt-5-mini),ai (gpt-5-mini)
2,哪個軟體被用來合成講師的聲音？,['也是VO3自己生成的\n他產生的影片是帶有聲音的\n那其他中文講課的聲音也通通都是合成的...,答案：Eleven Labs。,ai (gpt-5-mini),ai (gpt-5-mini)


In [18]:
df.to_json(
    os.path.join('data', 'deliverables', 'lee_course0_rag_dataset.json'),
    orient="records",
    force_ascii=False,
    indent=2
)